In [1]:
import netCDF4 as nc
import xarray as xr
import numpy as np
import os
import requests
from bs4 import BeautifulSoup
import json

# Get filenames

In [2]:
def get_links(url):
   response = requests.get(url)
   response.raise_for_status()
   soup = BeautifulSoup(response.content, 'html.parser')
   links = soup.find_all('a')
   return links

In [4]:
def get_files()

    to_ignore = ['ebasNRT.html',
             '/thredds/catalog.html',
             'https://nilu.no',
             '/thredds/serverInfo.html',
             'https://docs.unidata.ucar.edu/tds/4.6/adminguide/',
             'catalog.html?dataset=STATIONS']
    
    files = []
    url = 'https://thredds.nilu.no/thredds/catalog/stations/catalog.html'
    links = get_links(url)
    for a_tag in links:
        link = a_tag.get('href')
        if link not in to_ignore:
            station = link.replace('/catalog.html', '')
            url = f'https://thredds.nilu.no/thredds/catalog/stations/{station}/catalog.html'
            links = get_links(url)
            for a_tag in links:
                link = a_tag.get('href')
                if link not in to_ignore and not link.startswith('catalog.html?dataset=STATIONS/'):
                    speci = link.replace('/catalog.html', '')
                    url = f'https://thredds.nilu.no/thredds/catalog/stations/{station}/{speci}/catalog.html'
                    links = get_links(url)
                    for a_tag in links:
                        link = a_tag.get('href')
                        if link not in to_ignore and not link.startswith('catalog.html?dataset=STATIONS/'):
                            date = link.replace('/catalog.html', '')
                            url = f'https://thredds.nilu.no/thredds/catalog/stations/{station}/{speci}/{date}/catalog.html'
                            links = get_links(url)
                            for a_tag in links:
                                link = a_tag.get('href')
                                if link not in to_ignore and not link.startswith('catalog.html?dataset=STATIONS/'):
                                    lev = link.replace('/catalog.html', '')
                                    url = f'https://thredds.nilu.no/thredds/catalog/stations/{station}/{speci}/{date}/{lev}/catalog.html'
                                    links = get_links(url)
                                    for a_tag in links:
                                        link = a_tag.get('href')
                                        if link not in to_ignore and link != f'catalog.html?dataset=STATIONS/{station}/{speci}/{date}/{lev}':
                                            data = link.replace(f'catalog.html?dataset=STATIONS/{station}/{speci}/{date}/{lev}/', '')
                                            files.append(data)
                                            

    with open('ebas_nrt.json', 'w') as f:
        json.dump(files, f)

In [7]:
with open('ebas_nrt.json') as f:
    files = json.load(f)

In [10]:
wrong_files = []
correct_files = []
actris_nrt_path = 'https://thredds.nilu.no/thredds/stations/'
for file in files[0:10]:
    variables = []
    try:
        opendap_url = f'{actris_nrt_path}/{file}'
        ds = xr.open_dataset(opendap_url)
        correct_files.append(file)
    except Exception as error:
        wrong_files.append(file)

print(f'Total number of files: {len(files)}')
print(f'OK: {len(correct_files)}')
print(f'Error: {len(wrong_files)}')

syntax error, unexpected WORD_WORD, expecting SCAN_ATTR or SCAN_DATASET or SCAN_ERROR
context: <html^ xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" lang="en"> <head> <meta http-equiv="Content-Type" content="text/html; charset=UTF-8"> <title>TDS - Error report</title> <link rel="stylesheet" href="/thredds/tds.css" type="text/css"/> </head> <body> <h1>HTTP Status 404 - Not Found</h1> <HR size="1" noshade="noshade"> <p><b>Status</b> 404 - Not Found</p> <HR size="1" noshade="noshade"> <h3>THREDDS Data Server Version 4.6 -- <a href='https://www.unidata.ucar.edu/software/thredds/v4.6/tds/TDS.html'>Documentation</a></h3> </body></html>
syntax error, unexpected WORD_WORD, expecting SCAN_ATTR or SCAN_DATASET or SCAN_ERROR
context: <html^ xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" lang="en"> <head> <meta http-equiv="Content-Type" content="text/html; charset=UTF-8"> <title>TDS - Error report</title> <link rel="stylesheet" href="/thredds/tds.css" type="text/css"/> </head> <body> <h1>H

Total number of files: 315694
OK: 0
Error: 10


syntax error, unexpected WORD_WORD, expecting SCAN_ATTR or SCAN_DATASET or SCAN_ERROR
context: <html^ xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" lang="en"> <head> <meta http-equiv="Content-Type" content="text/html; charset=UTF-8"> <title>TDS - Error report</title> <link rel="stylesheet" href="/thredds/tds.css" type="text/css"/> </head> <body> <h1>HTTP Status 404 - Not Found</h1> <HR size="1" noshade="noshade"> <p><b>Status</b> 404 - Not Found</p> <HR size="1" noshade="noshade"> <h3>THREDDS Data Server Version 4.6 -- <a href='https://www.unidata.ucar.edu/software/thredds/v4.6/tds/TDS.html'>Documentation</a></h3> </body></html>
